In [ ]:
from __future__ import print_function, division
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import datetime

# GPU settings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# get the base graph and generator matrix
code_PCM0 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph/BaseGraph2_Set0.txt", int, delimiter='	')
code_PCM1 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph/BaseGraph2_Set1.txt", int, delimiter='	')
code_PCM2 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph/BaseGraph2_Set2.txt", int, delimiter='	')
code_GM_16 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph_GM/LDPC_GM_BG2_16.txt", int, delimiter=',')
code_GM_3 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph_GM/LDPC_GM_BG2_3.txt", int, delimiter=',')
code_GM_10 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph_GM/LDPC_GM_BG2_10.txt", int, delimiter=',')
code_GM_6 = np.loadtxt("/kaggle/input/hls-project/Neural-Protograph-LDPC-Decoding-main/BaseGraph_GM/LDPC_GM_BG2_6.txt", int, delimiter=',')
code_PCM = code_PCM0.copy()
Ldpc_PCM = [code_PCM0, code_PCM1, code_PCM2, code_PCM1]# four LDPC codes with different code lengths
Ldpc_GM = [code_GM_16, code_GM_3, code_GM_10, code_GM_6]
Z_array = np.array([16, 3, 10, 6])

In [ ]:
N = 52
m = 42
code_n = N
code_k = N - m
for i in range(0, code_PCM.shape[0]):
    for j in range(0, code_PCM.shape[1]):
        if (code_PCM[i, j] == -1):
            code_PCM[i, j] = 0
        else:
            code_PCM[i, j] = 1

In [ ]:
# network hyper-parameters
iters_max = 25     # number of iterations
sum_edge_c = np.sum(code_PCM, axis=1)
sum_edge_v = np.sum(code_PCM, axis=0)
sum_edge = np.sum(sum_edge_v)
neurons_per_even_layer = neurons_per_odd_layer = np.sum(sum_edge_v)
input_output_layer_size = N

# init the AWGN #
code_rate = 1.0 * (N - m) / (N-2)
# train SNR
SNR_Matrix = np.array([[9.0,6.05,4.1,2.95,2.25,1.8,1.55,1.3,1.15,1.05,0.94,0.85,0.83,0.81,0.8,0.8,0.8,0.75,0.75,0.7,0.7,0.7,0.7,0.7,0.7],
                       [9.1,6.2,4.6,3.7,3.2,3.0,2.8,2.7,2.6,2.55,2.5,2.45,2.4,2.4,2.4,2.35,2.35,2.3,2.3,2.3,2.25,2.25,2.25,2.25,2.25],
                       [9,6.05,4.1,3,2.4,2,1.7,1.5,1.4,1.4,1.3,1.3,1.2,1.2,1.2,1.2,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1,1],
                       [9.0,6.1,4.25,3.2,2.6,2.25,2,1.9,1.8,1.7,1.7,1.65,1.6,1.6,1.55,1.55,1.5,1.5,1.5,1.45,1.45,1.4,1.4,1.4,1.4]])
SNR_lin = 10.0 ** (SNR_Matrix / 10.0)
SNR_sigma = np.sqrt(1.0 / (2.0 * SNR_lin * code_rate))
# ramdom seed
word_seed = 2042
noise_seed = 1074
wordRandom = np.random.RandomState(word_seed)  # word seed
noiseRandom = np.random.RandomState(noise_seed)  # noise seed

In [ ]:
# train settings
learning_rate = 0.001
train_on_zero_word = True
numOfWordSim_train = 50
batch_size = numOfWordSim_train
num_of_batch = 50000

#get train samples
def create_mix_epoch(scaling_factor, wordRandom, noiseRandom, numOfWordSim, code_n, code_k, Z, code_GM, is_zeros_word):
    X = np.zeros([1, code_n * Z], dtype=np.float32)
    Y = np.zeros([1, code_n * Z], dtype=np.int64)

    # build set for epoch
    for sf_i in scaling_factor:
        if is_zeros_word:
            infoWord_i = 0 * wordRandom.randint(0, 2, size=(numOfWordSim, code_k * Z))
        else:
            infoWord_i = wordRandom.randint(0, 2, size=(numOfWordSim, code_k * Z))

        Y_i = np.dot(infoWord_i, code_GM) % 2
        X_p_i = noiseRandom.normal(0.0, 1.0, Y_i.shape) * sf_i + (-1) ** (1 - Y_i)  # pay attention to this 1->1 0->-1
        x_llr_i = 2 * X_p_i / ((sf_i) ** 2)  # defined as p1/p0
        X = np.vstack((X, x_llr_i))
        Y = np.vstack((Y, Y_i))
    X = X[1:]
    Y = Y[1:]
    X = np.reshape(X, [batch_size, code_n, Z])
    return X, Y

In [ ]:
# calculate ber and fer
def calc_ber_fer(snr_db, Y_test_pred, Y_test, numOfWordSim):
    ber_test = np.zeros(snr_db.shape[0])
    fer_test = np.zeros(snr_db.shape[0])
    for i in range(0, snr_db.shape[0]):
        Y_test_pred_i = Y_test_pred[i * numOfWordSim:(i + 1) * numOfWordSim, :]
        Y_test_i = Y_test[i * numOfWordSim:(i + 1) * numOfWordSim, :]
        ber_test[i] = np.abs(((Y_test_pred_i > 0) - Y_test_i)).sum() / (Y_test_i.shape[0] * Y_test_i.shape[1])
        fer_test[i] = (np.abs(((Y_test_pred_i > 0) - Y_test_i)).sum(axis=1) > 0).sum() * 1.0 / Y_test_i.shape[0]
    return ber_test, fer_test


In [ ]:
############################     init the connecting matrix between network layers   #################################
Lift_Matrix1 = []
Lift_Matrix2 = []
W_odd2even = np.zeros((sum_edge, sum_edge), dtype=np.float32)
W_skipconn2even = np.zeros((N, sum_edge), dtype=np.float32)
W_even2odd = np.zeros((sum_edge, sum_edge), dtype=np.float32)
W_output = np.zeros((sum_edge, N), dtype=np.float32)

In [ ]:
# init lifting matrix for cyclic shift
for t in range(0, 4, 1):
    Lift_M1 = np.zeros((neurons_per_odd_layer * Z_array[t], neurons_per_odd_layer * Z_array[t]), np.float32)
    Lift_M2 = np.zeros((neurons_per_odd_layer * Z_array[t], neurons_per_odd_layer * Z_array[t]), np.float32)
    code_PCM1 = Ldpc_PCM[t]
    k = 0
    for j in range(0, code_PCM1.shape[1]):
        for i in range(0, code_PCM1.shape[0]):
            if (code_PCM1[i, j] != -1):
                Lift_num = code_PCM1[i, j] % Z_array[t]
                for h in range(0, Z_array[t], 1):
                    Lift_M1[k * Z_array[t] + h, k * Z_array[t] + (h + Lift_num) % Z_array[t]] = 1
                k = k + 1
    k = 0
    for i in range(0, code_PCM1.shape[0]):
        for j in range(0, code_PCM1.shape[1]):
            if (code_PCM1[i, j] != -1):
                Lift_num = code_PCM1[i, j] % Z_array[t]
                for h in range(0, Z_array[t], 1):
                    Lift_M2[k * Z_array[t] + h, k * Z_array[t] + (h + Lift_num) % Z_array[t]] = 1
                k = k + 1
    Lift_Matrix1.append(Lift_M1)
    Lift_Matrix2.append(Lift_M2)

In [ ]:
# init W_odd2even  variable node updating
k = 0
vec_tmp = np.zeros((sum_edge), dtype=np.float32)  # even layer index read with column
for j in range(0, code_PCM.shape[1], 1):  # run over the columns
    for i in range(0, code_PCM.shape[0], 1):  # break after the first one
        if (code_PCM[i, j] == 1):  # finding the first one is ok
            num_of_conn = int(np.sum(code_PCM[:, j]))  # get the number of connection of the variable node
            idx = np.argwhere(code_PCM[:, j] == 1)  # get the indexes
            for l in range(0, num_of_conn, 1):  # adding num_of_conn columns to W
                vec_tmp = np.zeros((sum_edge), dtype=np.float32)
                for r in range(0, code_PCM.shape[0], 1):  # adding one to the right place
                    if (code_PCM[r, j] == 1 and idx[l][0] != r):
                        idx_row = np.cumsum(code_PCM[r, 0:j + 1])[-1] - 1
                        odd_layer_node_count = 0
                        if r > 0:
                            odd_layer_node_count = np.cumsum(sum_edge_c[0:r])[-1]
                        vec_tmp[idx_row + odd_layer_node_count] = 1  # offset index adding
                W_odd2even[:, k] = vec_tmp.transpose()
                k += 1
            break



In [ ]:
# init W_even2odd  parity check node updating
k = 0
for j in range(0, code_PCM.shape[1], 1):
    for i in range(0, code_PCM.shape[0], 1):
        if (code_PCM[i, j] == 1):
            idx_row = np.cumsum(code_PCM[i, 0:j + 1])[-1] - 1
            idx_col = np.cumsum(code_PCM[0: i + 1, j])[-1] - 1
            odd_layer_node_count_1 = 0
            odd_layer_node_count_2 = np.cumsum(sum_edge_c[0:i + 1])[-1]
            if i > 0:
                odd_layer_node_count_1 = np.cumsum(sum_edge_c[0:i])[-1]
            W_even2odd[k, odd_layer_node_count_1:odd_layer_node_count_2] = 1.0
            W_even2odd[k, odd_layer_node_count_1 + idx_row] = 0.0
            k += 1  # k is counted in column direction



In [ ]:
# init W_output odd to output
k = 0
for j in range(0, code_PCM.shape[1], 1):
    for i in range(0, code_PCM.shape[0], 1):
        if (code_PCM[i, j] == 1):
            idx_row = np.cumsum(code_PCM[i, 0:j + 1])[-1] - 1
            idx_col = np.cumsum(code_PCM[0: i + 1, j])[-1] - 1
            odd_layer_node_count = 0
            if i > 0:
                odd_layer_node_count = np.cumsum(sum_edge_c[0:i])[-1]
            W_output[odd_layer_node_count + idx_row, k] = 1.0
    k += 1



In [ ]:
# init W_skipconn2even  channel input
k = 0
for j in range(0, code_PCM.shape[1], 1):
    for i in range(0, code_PCM.shape[0], 1):
        if (code_PCM[i, j] == 1):
            W_skipconn2even[j, k] = 1.0
            k += 1

In [ ]:
##############################  bulid neural networks ############################
net_dict = {}
# init the learnable network parameters
Weights_Var = 0.5 * np.ones(sum_edge, dtype=np.float32)
Biases_Var = np.zeros(sum_edge, dtype=np.float32)
for i in range(0, iters_max, 1):
    net_dict["Weights_Var{0}".format(i)] = tf.Variable(Weights_Var.copy(), name="Weights_Var".format(i))
    net_dict["Biases_Var{0}".format(i)] = tf.Variable(Biases_Var.copy(), name="Biases_Var".format(i))

# the decoding neural network of Z=16
Z = 16
xa = tf.placeholder(tf.float32, shape=[batch_size, N, Z], name='xa')
ya = tf.placeholder(tf.float32, shape=[batch_size, N * Z], name='ya')
xa_input = tf.transpose(xa, [0, 2, 1])
net_dict["LLRa{0}".format(0)] = tf.zeros((batch_size, Z, sum_edge), dtype=tf.float32)

In [ ]:
import time
import os

os.makedirs('/kaggle/working/Weights_Var', exist_ok=True)
os.makedirs('/kaggle/working/Biases_Var', exist_ok=True)


In [ ]:
for i in range(0, iters_max, 1):
    #variable node update
    x0 = tf.matmul(xa_input, W_skipconn2even)
    x1 = tf.matmul(net_dict["LLRa{0}".format(i)], W_odd2even)
    x2 = tf.add(x0, x1)
    x2 = tf.transpose(x2, [0, 2, 1])
    x2 = tf.reshape(x2, [batch_size, neurons_per_odd_layer * Z])
    x2 = tf.matmul(x2, Lift_Matrix1[0].transpose())
    x2 = tf.reshape(x2, [batch_size, neurons_per_odd_layer, Z])
    x2 = tf.transpose(x2, [0, 2, 1])
    x_tile = tf.tile(x2, multiples=[1, 1, neurons_per_odd_layer])
    W_input_reshape = tf.reshape(W_even2odd.transpose(), [-1])
    #check node update
    x_tile_mul = tf.multiply(x_tile, W_input_reshape)
    x2_1 = tf.reshape(x_tile_mul, [batch_size, Z, neurons_per_odd_layer, neurons_per_odd_layer])
    x2_abs = tf.add(tf.abs(x2_1), 10000 * (1 - tf.to_float(tf.abs(x2_1) > 0)))
    x3 = tf.reduce_min(x2_abs, axis=3)
    x2_2 = -x2_1
    x4 = tf.add(tf.zeros((batch_size, Z, neurons_per_odd_layer, neurons_per_odd_layer)), 1 - 2 * tf.to_float(x2_2 < 0))
    x4_prod = -tf.reduce_prod(x4, axis=3)
    x_output_0 = tf.multiply(x3, tf.sign(x4_prod))
    x_output_0 = tf.transpose(x_output_0, [0, 2, 1])
    x_output_0 = tf.reshape(x_output_0, [batch_size, Z * neurons_per_odd_layer])
    x_output_0 = tf.matmul(x_output_0, Lift_Matrix2[0])
    x_output_0 = tf.reshape(x_output_0, [batch_size, neurons_per_odd_layer, Z])
    x_output_0 = tf.transpose(x_output_0, [0, 2, 1])
    # add learnable parameters
        # ===================== Dynamic Weight Sharing (DWS) ======================
    
    # Step 1: Detect satisfied / unsatisfied check nodes
    # Compute parity satisfaction using the sign product
    CN_sign = tf.sign(x_output_0)  # sign of messages from each CN
    CN_satisfied = tf.cast(tf.equal(tf.reduce_prod(CN_sign, axis=2), 1), tf.float32)  # parity satisfied -> 1
    CN_unsatisfied = 1.0 - CN_satisfied  # parity unsatisfied -> 1
    
    # Step 2: Define dynamic weights for SCNs and UCNs per iteration
    net_dict["Weights_SCN{0}".format(i)] = tf.Variable(
        0.5 * np.ones(neurons_per_odd_layer, dtype=np.float32),
        name="Weights_SCN{0}".format(i))
    net_dict["Weights_UCN{0}".format(i)] = tf.Variable(
        0.5 * np.ones(neurons_per_odd_layer, dtype=np.float32),
        name="Weights_UCN{0}".format(i))
    
    # Step 3: Apply corresponding weights dynamically
    # |x_output_0| gives magnitude of CN messages
    x_output_SCN = tf.multiply(tf.abs(x_output_0), net_dict["Weights_SCN{0}".format(i)])
    x_output_UCN = tf.multiply(tf.abs(x_output_0), net_dict["Weights_UCN{0}".format(i)])
    
    # broadcast CN_satisfied/unsatisfied along last dimension (Z axis)
    x_output_1 = (x_output_SCN * tf.expand_dims(CN_satisfied, -1) +
                  x_output_UCN * tf.expand_dims(CN_unsatisfied, -1))
    
    # Add bias as before
    x_output_1 = tf.add(x_output_1, net_dict["Biases_Var{0}".format(i)])
    
    # ReLU nonlinearity
    x_output_1 = tf.multiply(x_output_1, tf.to_float(x_output_1 > 0))
    
    # Update LLR for next iteration
    net_dict["LLRa{0}".format(i+1)] = tf.multiply(x_output_1, tf.sign(x_output_0))
    # ========================================================================

    # output
    y_output_2 = tf.matmul(net_dict["LLRa{0}".format(i+1)], W_output)
    y_output_3 = tf.transpose(y_output_2, [0, 2, 1])
    y_output_4 = tf.add(xa, y_output_3)
    net_dict["ya_output{0}".format(i)] = tf.reshape(y_output_4, [batch_size, N * Z], name='ya_output'.format(i))
    # calculate loss
    net_dict["lossa{0}".format(i)] = 1.0 * tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=ya,
                                                            logits=net_dict["ya_output{0}".format(i)]), name='lossa'.format(i))
    # AdamOptimizer
    net_dict["train_stepa{0}".format(i)] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(
    net_dict["lossa{0}".format(i)],
    var_list=[net_dict["Weights_SCN{0}".format(i)],
              net_dict["Weights_UCN{0}".format(i)],
              net_dict["Biases_Var{0}".format(i)]])



##################################  Train  ####################################
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [ ]:

# for i in range(0, iters_max, 1):
#     #variable node update
#     x0 = tf.matmul(xa_input, W_skipconn2even)
#     x1 = tf.matmul(net_dict["LLRa{0}".format(i)], W_odd2even)
#     x2 = tf.add(x0, x1)
#     x2 = tf.transpose(x2, [0, 2, 1])
#     x2 = tf.reshape(x2, [batch_size, neurons_per_odd_layer * Z])
#     x2 = tf.matmul(x2, Lift_Matrix1[0].transpose())
#     x2 = tf.reshape(x2, [batch_size, neurons_per_odd_layer, Z])
#     x2 = tf.transpose(x2, [0, 2, 1])
#     x_tile = tf.tile(x2, multiples=[1, 1, neurons_per_odd_layer])
#     W_input_reshape = tf.reshape(W_even2odd.transpose(), [-1])
#     #check node update
#     x_tile_mul = tf.multiply(x_tile, W_input_reshape)
#     x2_1 = tf.reshape(x_tile_mul, [batch_size, Z, neurons_per_odd_layer, neurons_per_odd_layer])
#     x2_abs = tf.add(tf.abs(x2_1), 10000 * (1 - tf.to_float(tf.abs(x2_1) > 0)))
#     x3 = tf.reduce_min(x2_abs, axis=3)
#     x2_2 = -x2_1
#     x4 = tf.add(tf.zeros((batch_size, Z, neurons_per_odd_layer, neurons_per_odd_layer)), 1 - 2 * tf.to_float(x2_2 < 0))
#     x4_prod = -tf.reduce_prod(x4, axis=3)
#     x_output_0 = tf.multiply(x3, tf.sign(x4_prod))
#     x_output_0 = tf.transpose(x_output_0, [0, 2, 1])
#     x_output_0 = tf.reshape(x_output_0, [batch_size, Z * neurons_per_odd_layer])
#     x_output_0 = tf.matmul(x_output_0, Lift_Matrix2[0])
#     x_output_0 = tf.reshape(x_output_0, [batch_size, neurons_per_odd_layer, Z])
#     x_output_0 = tf.transpose(x_output_0, [0, 2, 1])
#     # ===================== Dynamic Weight Sharing (DWS) - CORRECT ======================

#     # At this point x_output_0 has shape [batch_size, Z, neurons_per_odd_layer]
#     # We'll compute the per-edge UCN mask using VN APP signs (same logic as Main_Functions).
    
#     # --- 1) Compute current APP for variable nodes (same as original code's y_output_4)
#     # Note: y_output_4 is computed later in your original code; we can compute temporary VN_APP
#     # from current LLRs: current APP = xa + sum(C->V messages)
#     # We already have x_output_0 -> next LLRs are computed after weighting; but to follow Main_Functions
#     # we use the previous iteration's APP. To be consistent, use the current y_output_4 (which is computed below in original).
#     # For safety here, compute VN_APP from available tensors:
#     # Build tentative APP as in original: sum incoming C->V (x_output_0 with sign) plus xa_input
#     temp_LLR = tf.multiply(tf.abs(x_output_0), tf.sign(x_output_0))  # this is |msg| * sign = original msg (shape [B,Z,E])
#     # Sum C->V into VN domain via Lift_Matrix2 and W_output path:
#     # reshape-> matmul by Lift_Matrix2 -> reshape -> sum with W_output
#     tmp = tf.transpose(temp_LLR, [0, 2, 1])                         # [B, E, Z]
#     tmp = tf.reshape(tmp, [batch_size, Z * neurons_per_odd_layer])   # [B, Z*E]
#     tmp = tf.matmul(tmp, Lift_Matrix2[0])                            # permute -> [B, Z*E]
#     tmp = tf.reshape(tmp, [batch_size, neurons_per_odd_layer, Z])    # [B, E, Z]
#     tmp = tf.transpose(tmp, [0, 2, 1])                               # [B, Z, E]
#     y_tmp = tf.matmul(tmp, W_output)                                 # [B, Z, N]  (since W_output was [E, N])
#     y_tmp = tf.transpose(y_tmp, [0, 2, 1])                           # [B, N, Z]
#     VN_APP = tf.add(xa, y_tmp)                                       # [B, N, Z]  tentative APP
    
#     # Follow Main_Functions path to compute per-edge UCN mask from VN_APP
#     VN_APP_neg = -VN_APP                              # same sign convention as Main_Functions
#     VN_APP_sign = tf.add(tf.to_float(VN_APP_neg > 0), -tf.to_float(VN_APP_neg <= 0))  # [B,N,Z] in {+1,-1}
    
#     # Map VN signs to edges
#     VN_APP_sign_edge = tf.matmul(VN_APP_sign, W_skipconn2even)   # [B, N, Z] x [N, E] -> [B, Z, E(V)] (thanks to numpy matmul broadcasting)
#     VN_APP_sign_edge = tf.transpose(VN_APP_sign_edge, [0, 2, 1])  # [B, E, Z]
#     VN_APP_sign_edge = tf.reshape(VN_APP_sign_edge, [batch_size, neurons_per_odd_layer * Z])  # [B, E*Z]
    
#     # Permute to CN inputs
#     CN_in_sign = tf.matmul(VN_APP_sign_edge, Lift_Matrix1[0].transpose())   # [B, E*Z]
#     CN_in_sign = tf.reshape(CN_in_sign, [batch_size, neurons_per_odd_layer, Z])  # [B, E, Z]
#     CN_in_sign = tf.transpose(CN_in_sign, [0, 2, 1])  # [B, Z, E]
    
#     # Build tile and compute sign product across edges of each CN using adjacency (same trick)
#     CN_in_sign_tile = tf.tile(CN_in_sign, multiples=[1, 1, neurons_per_odd_layer])  # [B,Z,E,E]
#     CN_in_sign_tile = tf.multiply(CN_in_sign_tile, tf.reshape(W_even2odd_with_self.transpose(), [-1]))
#     CN_in_sign_tile = tf.reshape(CN_in_sign_tile, [batch_size, Z, neurons_per_odd_layer, neurons_per_odd_layer])
#     CN_in_sign_tile = tf.add(CN_in_sign_tile, 1.0 * (1 - tf.to_float(tf.abs(CN_in_sign_tile) > 0)))  # avoid zeros
#     CN_sign_edge = tf.reduce_prod(CN_in_sign_tile, axis=3)  # [B,Z,E] product across neighbors -> sign per CN-edge
#     UCN_idx_edge = tf.to_float(CN_sign_edge < 0)            # [B,Z,E] 1 for unsatisfied, 0 for satisfied
    
#     # Convert shape to match message tensors ordering (we already have [B,Z,E])
#     SCN_idx_edge = 1.0 - UCN_idx_edge                       # satisfied mask
    
#     # --- 2) Define DWS trainable weights (per edge dimension) for this iteration
#     # Use same shape as original Weights_Var / Biases (E elements)
#     w_scn_name = "Weights_SCN{0}".format(i)
#     w_ucn_name = "Weights_UCN{0}".format(i)
#     bias_name  = "Biases_Var{0}".format(i)
    
#     # create variables if not already in net_dict
#     if w_scn_name not in net_dict:
#         net_dict[w_scn_name] = tf.Variable(0.5 * np.ones(neurons_per_odd_layer, dtype=np.float32), name=w_scn_name)
#     if w_ucn_name not in net_dict:
#         net_dict[w_ucn_name] = tf.Variable(0.5 * np.ones(neurons_per_odd_layer, dtype=np.float32), name=w_ucn_name)
#     # Bias already exists in your code as Biases_Var{i}
    
#     # --- 3) Apply weights per-edge using masks
#     # x_output_0 is [B, Z, E]
#     x_abs = tf.abs(x_output_0)  # [B, Z, E]
#     W_scn = net_dict[w_scn_name]  # shape [E]
#     W_ucn = net_dict[w_ucn_name]
    
#     # reshape weights to broadcast: [1,1,E]
#     W_scn_b = tf.reshape(W_scn, [1, 1, -1])
#     W_ucn_b = tf.reshape(W_ucn, [1, 1, -1])
    
#     x_output_SCN = x_abs * W_scn_b
#     x_output_UCN = x_abs * W_ucn_b
    
#     # Apply masks
#     x_output_1 = x_output_SCN * SCN_idx_edge + x_output_UCN * UCN_idx_edge
    
#     # Add bias (bias vector has length E, broadcast to [B,Z,E])
#     bias_b = tf.reshape(net_dict[bias_name], [1, 1, -1])
#     x_output_1 = x_output_1 + bias_b
    
#     # ReLU and restore sign
#     x_output_1 = x_output_1 * tf.to_float(x_output_1 > 0)
#     net_dict["LLRa{0}".format(i+1)] = x_output_1 * tf.sign(x_output_0)
    
#     # ========================================================================

#     # output
#     y_output_2 = tf.matmul(net_dict["LLRa{0}".format(i+1)], W_output)
#     y_output_3 = tf.transpose(y_output_2, [0, 2, 1])
#     y_output_4 = tf.add(xa, y_output_3)
#     net_dict["ya_output{0}".format(i)] = tf.reshape(y_output_4, [batch_size, N * Z], name='ya_output'.format(i))
#     # calculate loss
#     net_dict["lossa{0}".format(i)] = 1.0 * tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=ya,
#                                                             logits=net_dict["ya_output{0}".format(i)]), name='lossa'.format(i))
#     # AdamOptimizer
#     net_dict["train_stepa{0}".format(i)] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(
#     net_dict["lossa{0}".format(i)],
#     var_list=[net_dict["Weights_SCN{0}".format(i)],
#               net_dict["Weights_UCN{0}".format(i)],
#               net_dict["Biases_Var{0}".format(i)]])



# #################################  Train  ####################################
# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()

In [ ]:
for iter in range(0, iters_max, 1):
    for i in range(0, num_of_batch, 1):
        #ramdom choose Z with different probabilities
        index = [0, 1, 2, 3, 0, 0]
        Z = 16
        SNR_set = np.array([SNR_sigma[0, iter]])
        training_received_data, training_coded_bits = create_mix_epoch(SNR_set, wordRandom, noiseRandom, numOfWordSim_train,
                                                                        code_n, code_k, Z,
                                                                        Ldpc_GM[0],
                                                                        train_on_zero_word)
        training_labels_for_mse = training_coded_bits
        y_pred, train_loss, _ = sess.run(fetches=[net_dict["ya_output{0}".format(iter)], net_dict["lossa{0}".format(iter)],
                                                    net_dict["train_stepa{0}".format(iter)]],
                                            feed_dict={xa: training_received_data, ya: training_labels_for_mse})
        if i % 200 == 0:
            print('iteration: [{0}/{1}]\t'
                    'epoch: [{2}/{3}]\t'
                    'loss: {4}\t'.format(
                iter + 1, iters_max, i, num_of_batch, train_loss))

    ##################################  save weights and biases  ####################################
    a, b = sess.run(fetches=[net_dict["Weights_Var{0}".format(iter)], net_dict["Biases_Var{0}".format(iter)]])
    np.savetxt('/kaggle/working/Weights_Var/Weights_Var{0}.txt'.format(iter), a, fmt='%s', delimiter=',')
    np.savetxt('/kaggle/working/Biases_Var/Biases_Var{0}.txt'.format(iter), b, fmt='%s', delimiter=',')
